Таблица успешно создана в PostgreSQL
Соединение с PostgreSQL закрыто


In [5]:
import pandas as pd
df = pd.read_csv('table.csv')
df['time'] = pd.to_datetime(df['time'])
df.sort_values(by='time')
df['time'] = pd.to_datetime(df['time']) - pd.to_timedelta (7, unit='d')
cogort = df.groupby([pd.Grouper(key='time', freq='W')])['userId'].count()
lst = []
for i in range(len(cogort)):
    for k in range(cogort[i]):
        lst.append(i + 1)
df['week'] = lst
reg = df.loc[(df['eventName'] == 'register')][['userId', 'week']]
downl = df.loc[(df['eventName'] == 'download')][['userId', 'week']]
mrg = pd.merge(reg, downl, on='userId', how='inner')
mrg = mrg.loc[mrg['week_x'] == mrg['week_y']]
mrg = mrg.drop_duplicates()
both = mrg.groupby(['week_x'])['userId'].count().reset_index(name='both')
both = both.rename(columns={'week_x':'week'})
users = reg.groupby(['week'])['userId'].count().reset_index(name='reg')
cogorts = pd.DataFrame({'week': [i + 1 for i in range(len(cogort))]})
res1 = pd.merge(cogorts, users, on='week', how='left')
res2 =  pd.merge(res1, both, on='week', how='left')
res2 = res2.rename(columns={'reg':'users', 'both':'CR'})
res2['CR'] = round(100 * res2['CR'] / res2['users'], 2)
res2 = res2.rename(columns={'reg':'users'})
both.loc[both.groupby('week').mean(), 'mar'] = 1
print(both)

KeyError: "None of [Index([('b', 'o', 't', 'h')], dtype='object')] are in the [index]"